In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key

In [10]:
weather_data = "output_data/cities.csv"

weather_data_df = pd.read_csv(weather_data)
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Belyy Yar,0,RU,1586651646,85,53.60,91.39,23.00,2.24
1,Busselton,84,AU,1586651646,65,-33.65,115.33,72.00,9.33
2,Ushuaia,40,AR,1586651646,75,-54.80,-68.30,41.00,17.22
3,Kaitangata,96,NZ,1586651646,84,-46.28,169.85,55.00,3.00
4,Gumdag,0,TM,1586651646,60,39.21,54.59,53.58,4.94
...,...,...,...,...,...,...,...,...,...
564,Lewistown,1,US,1586651761,39,40.60,-77.57,50.00,6.93
565,Falmouth,4,GB,1586651618,76,50.15,-5.07,52.00,1.12
566,Tashigang,13,BT,1586651761,30,27.33,91.55,57.63,4.56
567,Lethem,78,GY,1586651761,55,3.38,-59.80,84.15,10.07


In [38]:
gmaps.configure(api_key=g_key)

locations = weather_data_df[["Lat", "Lng"]]   

humidity = weather_data_df["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
new_weather_data_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness", "Country", "Lat", "Lng"])

max_temp = (new_weather_data_df["Max Temp"] <= 80) & (new_weather_data_df["Max Temp"] > 70)
wind_speed = (new_weather_data_df["Wind Speed"] < 10)
cloudiness = (new_weather_data_df["Cloudiness"] == 0)

new_weather_data_df = new_weather_data_df[max_temp & wind_speed & cloudiness]
new_weather_data_df

,City,Max Temp,Wind Speed,Cloudiness,Country,Lat,Lng
385,Sorbhog,71.58,6.06,0,IN,26.50,90.87
431,Chākuliā,76.91,2.91,0,IN,22.48,86.72
525,Tura,71.28,4.85,0,IN,25.52,90.22
550,Deori,79.34,2.77,0,IN,21.45,82.62


In [22]:
hotel_df = pd.DataFrame(new_weather_data_df,columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
385,Sorbhog,IN,26.50,90.87,
431,Chākuliā,IN,22.48,86.72,
525,Tura,IN,25.52,90.22,
550,Deori,IN,21.45,82.62,


In [24]:
target_coordinates = f"{hotel_df['Lat']},{hotel_df['Lng']}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

parameters = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=parameters)

In [25]:
hotel_name = []

parameters = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    parameters["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
  
    name_address = requests.get(base_url, params=parameters)    
   
    name_address = name_address.json()
  
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

hotel_name

[nan, 'Debi Rani Debi Lodge', 'Hotel Polo Orchid', nan]

In [45]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
385,Sorbhog,IN,26.50,90.87,NaN
431,Chākuliā,IN,22.48,86.72,Debi Rani Debi Lodge
525,Tura,IN,25.52,90.22,Hotel Polo Orchid
550,Deori,IN,21.45,82.62,NaN


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [47]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))